In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from skimage import data, io, filters
from skimage.color import rgb2hsv, hsv2rgb, xyz2rgb
from skimage import data, img_as_float
from skimage import exposure

from lippmann import show_spectrum
from display_spectral_data import load_specim_data
from color_tools import upsample_hue_saturation, from_spectrum_to_xyz, from_xyz_to_rgb
from spectrum import Spectrum3D


### Warning: data has to be downloaded from SWITCH drive (shared by Arnaud)

In [ ]:
name = "parrot" # prefix of the file with data and results 

# directory = "visible_only"
# directory = "full_estimate_depth"
directory = 0 # name of the directory in which the results are stored (number o string)

results_path = f"PNAS/{directory}/{name}"

c0 = 299792458
n0 = 1.5
c = c0 / n0

result = np.load(f"{results_path}.npy")
with open(f"{results_path}.pkl", "rb") as pickled_params:
    params = pickle.load(pickled_params)

if "downsampling" in params:
    ds = params["downsampling"]
else:
    ds = 25
    
# if the data is stored in "Cubes"     
downsampled, wavelengths = load_specim_data("Cubes/" + name, ds, cut=True)


if params["visible"]: #TODO what should be there? 
    omegas_est = np.linspace(2 * np.pi * c / 400E-9, 2 * np.pi * c / 700E-9, result.shape[2])
    print("Restricting spectrum to visible")
else:
    omegas_rec = 2 * np.pi * c / wavelengths
    omegas_est = np.linspace(np.max(omegas_rec), np.min(omegas_rec), result.shape[2])
wavelengths_est = 2 * np.pi * c / omegas_est

In [ ]:
print("Experiment setup")
print_params = params.copy()

if "estimate_depth" in params:
    if params["estimate_depth"]:
        plt.matshow(params["Z_estimates"])
        plt.colorbar()
        plt.savefig(f"{results_path}_z.pdf")
        plt.show()
        print_params.pop("Z_estimates")
print(print_params)

In [ ]:
# plt.figure(figsize=(5, 5))
# plt.plot(wavelengths, downsampled.reshape((-1, 204)).T)
# plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

show_spectrum(wavelengths, downsampled[7, 7, :], show_background=True, short_display=True, ax=ax1)
show_spectrum(wavelengths_est, result[7, 7, :], show_background=True, short_display=True, ax=ax2)
ax1.set_title("original spectrum")
title = f"Z: {params['Z']}, N: {params['N']}, Visible: {params['visible']}"
if "estimate_depth" in params and params["estimate_depth"]:
    title += f", Z est: {params['Z_estimates'][7, 7]:.2e}"
ax2.set_title(title)
plt.savefig(f"{results_path}_point.pdf")
plt.show()

### Question:
Why do we get negative numbers there?

In [ ]:
recorded_xyz = from_spectrum_to_xyz(wavelengths, downsampled, normalize=False)
recorded_xyz = recorded_xyz / np.min(np.sum(recorded_xyz, axis=2))
recorded_rgb = from_xyz_to_rgb(recorded_xyz)

estimated_xyz = from_spectrum_to_xyz(wavelengths_est, result, normalize=False)
estimated_xyz = estimated_xyz / np.min(np.sum(estimated_xyz, axis=2))
estimated_rgb = from_xyz_to_rgb(estimated_xyz)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(recorded_rgb)
ax2.imshow(estimated_rgb)
ax1.set_title("original colors")
ax2.set_title(title)
plt.savefig(f"{results_path}_low_res.pdf")
plt.show()

In [ ]:
image = io.imread("Cubes/" + name + ".png")
image = np.swapaxes(image[:, ::-1, :3], 1, 0)
cut_idx = np.loadtxt("Cubes/" + name + "_cut.txt").astype(np.int)
image = image[cut_idx[0, 0]:cut_idx[0, 1], cut_idx[1, 0]:cut_idx[1, 1]]
 
image = exposure.adjust_gamma(image, 0.4)
plt.figure(figsize=(5, 5))
plt.imshow(image)
plt.show()

In [ ]:
recorded_upsampled = upsample_hue_saturation(image, recorded_rgb, order=1)
estimated_upsampled = upsample_hue_saturation(image, estimated_rgb, order=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(recorded_upsampled)
ax2.imshow(estimated_upsampled)
plt.show()

In [ ]:
import spectrum_recovery_pool as recov

A = recov.generate_matrix_A(omegas_est, 5e-6, r=params["r"])

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

show_spectrum(wavelengths, downsampled[7, 7, :], show_background=True, short_display=True, ax=ax1)
show_spectrum(wavelengths_est, result[7, 7, :], show_background=True, short_display=True, ax=ax2)
show_spectrum(wavelengths_est, np.abs(A @ result[7, 7, :])**2, show_background=True, short_display=True, ax=ax3)